In [38]:
import astro as ast
import h5py
import os
import general as gen

In [39]:
map_file = './haslam408_ds_Remazeilles2014.fits'
galactic_coord_file = './pixel_coords_map_ring_galactic_res9.fits'
save_folder = '/mnt/d/sky_models/map_az_el_lst'
work_dir = '/home/christian/global21cm'

In [40]:
map_orig, lon, lat = ast.map_remazeilles_408MHz(map_file, galactic_coord_file)

REMAZEILLES 408MHz MAP


In [41]:
if not os.path.exists(save_folder) or not 'save_file_hdf5' in os.listdir(save_folder):
    LST, AZ, EL = ast.galactic_to_local_coordinates_24h_LST(lon, lat, INST_lat_deg= -26.714778, INST_lon_deg=116.605528, seconds_offset=359)
    os.makedirs(save_folder)
    os.chdir(save_folder)
    with h5py.File('save_file_hdf5', 'w') as hf:
        hf.create_dataset('LST', data = LST)
        hf.create_dataset('AZ',  data = AZ)
        hf.create_dataset('EL',  data = EL)
    os.chdir(work_dir)

In [42]:
freq_array_X, AZ_beam, EL_beam, Et_shifted, Ep_shifted, gain_shifted = gen.read_beam_FEKO('blade_dipole.out', 0)

 Frequency in Hz:               FREQ =    4.00000E+07

 Frequency in Hz:               FREQ =    4.00000E+07

 Frequency in Hz:               FREQ =    4.10000E+07

 Frequency in Hz:               FREQ =    4.10000E+07

 Frequency in Hz:               FREQ =    4.20000E+07

 Frequency in Hz:               FREQ =    4.20000E+07

 Frequency in Hz:               FREQ =    4.30000E+07

 Frequency in Hz:               FREQ =    4.30000E+07

 Frequency in Hz:               FREQ =    4.40000E+07

 Frequency in Hz:               FREQ =    4.40000E+07

 Frequency in Hz:               FREQ =    4.50000E+07

 Frequency in Hz:               FREQ =    4.50000E+07

 Frequency in Hz:               FREQ =    4.60000E+07

 Frequency in Hz:               FREQ =    4.60000E+07

 Frequency in Hz:               FREQ =    4.70000E+07

 Frequency in Hz:               FREQ =    4.70000E+07

 Frequency in Hz:               FREQ =    4.80000E+07

 Frequency in Hz:               FREQ =    4.80000E+07

 Frequency

 Frequency in Hz:               FREQ =    1.14000E+08

 Frequency in Hz:               FREQ =    1.15000E+08

 Frequency in Hz:               FREQ =    1.15000E+08

 Frequency in Hz:               FREQ =    1.16000E+08

 Frequency in Hz:               FREQ =    1.16000E+08

 Frequency in Hz:               FREQ =    1.17000E+08

 Frequency in Hz:               FREQ =    1.17000E+08

 Frequency in Hz:               FREQ =    1.18000E+08

 Frequency in Hz:               FREQ =    1.18000E+08

 Frequency in Hz:               FREQ =    1.19000E+08

 Frequency in Hz:               FREQ =    1.19000E+08

 Frequency in Hz:               FREQ =    1.20000E+08

 Frequency in Hz:               FREQ =    1.20000E+08

AZ_antenna_axis = 0 deg


In [ ]:
lst_az_el_file = save_folder+'/save_file_hdf5'

# Beam
beam_all_X = gain_shifted
FLOW         = 50
FHIGH        = 150
freq_array   = freq_array_X[(freq_array_X >= FLOW) & (freq_array_X <= FHIGH)]
beam_all     = beam_all_X[(freq_array_X >= FLOW) & (freq_array_X <= FHIGH), :, :]

# Sky model
map_freq = 408
sky_model = ast.map_power_law(map_orig, map_freq, lon, lat, freq_array, spectral_index_lat_model='step', lat_edge_deg=10, index_inband=2.5, index_outband=2.6, sigma_deg=8.5, index_center=2.4, index_pole=2.65)

# Local Coordinates
LST, AZ_lst, EL_lst = gen.read_hdf5_LST_AZ_EL(lst_az_el_file)

lst_out, freq_out, conv_out, ant_temp_out = ast.parallel_convolution(LST, freq_array, AZ_beam, EL_beam, beam_all, AZ_lst, EL_lst, sky_model, 2, normalization='yes', normalization_solid_angle_above_horizon_freq=1)

In [ ]:
with h5py.File(save_folder+'/save_parallel_convolution', 'w') as hf:
    hf.create_dataset('LST_out', data = lst_out)
    hf.create_dataset('freq_out',  data = freq_out)
    hf.create_dataset('conv_out',  data = conv_out)
    hf.create_dataset('ant_temp_out',  data = ant_temp_out)

To vary: AZ_antenna_axis in gen.read_beam_FEKO(), Ncpu (when using Niagara), INST_lat_deg in galactic_to_local()